<a href="https://colab.research.google.com/github/marta-manzin/agentic-shopping-assistant/blob/main/agentic_shopping_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛒 Agentic Shopping Assistant

This notebook will go through all the steps to create an agentic shopping assistant. \
We will:
1. Connect to OpenAI
2. Create a simple agent
3. Create an MCP server
4. Create an MCP client
5. Create a LangGraph agent

<br/>
<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/assistant.png" width="600">

# ⚙️ Setup

Setup
Before we start using OpenAI models, you need to set an API key. \
If you don't already have an key, you can generate one at: https://platform.openai.com/api-keys.

Save the key as a Colab Secret variable called "OPENAI_API_KEY":
1. Click on the key icon in the left bar menu.
2. Click on `+ Add new secret`.
3. Name the variable and paste the key in the value field.
4. Enable notebook access.

<br/>
<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/colab_setup.png" width="600">



Import the API key into the notebook.

In [ ]:
import os

# Detect if running in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Set API key based on environment
if IN_COLAB:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
else:
    # For local Jupyter: ensure OPENAI_API_KEY is set in your environment
    if "OPENAI_API_KEY" not in os.environ:
        print("Warning: OPENAI_API_KEY not found in environment variables")

Then, make a test call to OpenAI.

In [ ]:
import openai
client = openai.OpenAI()
model = "gpt-4o"

# Test that the LLM is set up correctly
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Say 'OK' if you can read this."}],
    max_tokens=10
)
print(f"LLM test: {response.choices[0].message.content}")

# 🤖 Creating an Agent

In Python, a set is an unordered collection of unique elements. \
We will build an agent that adds and removes strings from a set.

## Defining the tools

The System Prompt gives context to the LLM.

In [ ]:
SYSTEM_PROMPT = """
You are a helpful assistant that adds and removes strings from a set.

You have access to tools that let you:
1. Add a string, if it is not already in the set.
2. Remove a string.
3. Read all contents of the set.
"""

Here are the available tools:

In [ ]:
MY_SET = set()

def insertion_tool(s: str) -> str:
  """Tool: Add a string to a set."""
  try:
    MY_SET.add(s)
    return f"Inserted '{s}'."
  except Exception as ex:
    return f"Failed to insert '{s}'. {ex!r}"

def removal_tool(s: str) -> str:
  """Tool: Remove a string from a set."""
  try:
    if s in MY_SET:
      MY_SET.remove(s)
      return f"Removed '{s}'."
    else:
      return f"'{s}' is not in the set."
  except Exception as ex:
    return f"Failed to remove '{s}'. {ex!r}"

def get_set_tool() -> str:
  """Tool: Get the contents of the set."""
  try:
    if MY_SET:
      return f"The set contains: {sorted(MY_SET)}"
    else:
      return "The set is empty."
  except Exception as ex:
    return f"Failed to get set contents. {ex!r}"

tool_map = {
    "insertion_tool": insertion_tool,
    "removal_tool": removal_tool,
    "get_set_tool": get_set_tool
}

Provide a description of each tool to the LLM.

In [ ]:
tools: list[dict] = [
    {
        "type": "function",
        "function": {
            "name": "insertion_tool",
            "description": "Add a string to a set.",
            "parameters": {
                "type": "object",
                "properties": {
                    "s": {
                        "type": "string",
                        "description": "The string to be added."
                    },
                },
                "required": ["s"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "removal_tool",
            "description": "Remove a string from a set.",
            "parameters": {
                "type": "object",
                "properties": {
                    "s": {
                        "type": "string",
                        "description": "The string to be removed."
                    },
                },
                "required": ["s"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_set_tool",
            "description": "Get the contents of the set.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
]

## Calling a Tool

If the LLM decides to run a tool, instead of responding with a message, it will respond with a `tool_call` object.

In [ ]:
from types import SimpleNamespace

# Create a tool_call object that matches OpenAI's structure
tool_call = SimpleNamespace(
    id="call_abc123",
    function=SimpleNamespace(
        name="insertion_tool",
        arguments='{"s":"apple"}'
    ),
    type="function"
)

Extract the function name from the tool call.

In [ ]:
function_name = tool_call.function.name
function_name

Parse the arguments from JSON string to dictionary.

In [ ]:
import json

arguments = json.loads(tool_call.function.arguments)
arguments

Important! Verify that the function is one of the allowed tools.

In [ ]:
allowed_tool_names = [tool["function"]["name"] for tool in tools]
if function_name not in allowed_tool_names:
    print(f"Error: '{function_name}' is not an allowed tool.")
else:
    print(f"'{function_name}' is an allowed tool.")


Verify that the function exists and is callable.

In [ ]:
tool_func = tool_map[function_name]
if tool_func is None or not callable(tool_func):
    print(f"Unknown function: {function_name}")
else:
    print(f"Function {function_name} exists.")

Call the function with the unpacked arguments and print its response.

In [ ]:
MY_SET = set()

response = tool_func(**arguments)
response

Combine all tool calling steps in one method.

In [ ]:
import json

def execute(tool_call) -> str:
    """Execute a tool call and return the result, if any."""
    # Extract the function name from the tool call
    function_name = tool_call.function.name

    # Parse the arguments from JSON string to dictionary
    arguments = json.loads(tool_call.function.arguments)

    # Important! Verify that the function is one of the allowed tools
    allowed_tool_names = [tool["function"]["name"] for tool in tools]
    if function_name not in allowed_tool_names:
        return f"Error: '{function_name}' is not an allowed tool."

    # Verify that the function exists and is callable
    tool_func = tool_map[function_name]
    if tool_func is None or not callable(tool_func):
        return f"Unknown function: {function_name}"

    # Call the function with the unpacked arguments
    response = tool_func(**arguments)

    # Return the tool's response
    return response

## The agentic loop
Instead of using a ready-made framework, the code below implements *direct orchestration*.

We want to build a `while True` loop with tool calls when the LLM requests it.  

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow.png" width="600">
</br>
</br>

<img src="https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/main/images/agentic_flow_1.png" width="600">


`print_history` is a utility function that will display the LLM conversation history.

In [ ]:
import textwrap

def print_history(messages, width=70):
    """Pretty print messages history."""
    # For each message in the history
    for i, msg in enumerate(messages):
        print(f"\n[{i}] Role: {msg['role']}")

        # Display message text if present
        if msg.get('content'):
            for line in str(msg['content']).split('\n'):
                wrapped = textwrap.fill(line, width=width, initial_indent='    ', subsequent_indent='    ')
                print(wrapped)

        # Display tool calls if present
        if msg.get('tool_calls'):
            for tc in msg['tool_calls']:
                func_name = tc.function.name
                func_args = tc.function.arguments
                print(f"    🔧 {func_name}({func_args})")

        # If this is an assistant message with no tool calls, show completion
        if msg['role'] == 'assistant' and not msg.get('tool_calls'):
            print(f"\n⭐ The resulting set is: {MY_SET}")

Initialize the chat history.

In [ ]:
messages = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": "Please add 'apples' to the set."
    }
]

print_history(messages)

Ask the agent what to do next.

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
).choices[0].message

# Display the raw tool call in the agent's response
tool_calls_data = [tc.model_dump() for tc in response.tool_calls]
print(json.dumps(tool_calls_data, indent=2))

Update the chat history with the agent's response.

In [ ]:
messages.append({
    "role": "assistant",
    "content": response.content,
    "tool_calls": response.tool_calls
})

print_history(messages)

Execute the tool call.

In [ ]:
MY_SET = set()

outcome = execute(response.tool_calls[0])
outcome

Append the outcome to the message history.

In [ ]:
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(outcome)
})
print_history(messages)

Combine all direct orchestration steps into one method.

In [ ]:
def submit_request(
    user_prompt: str,
    verbose: bool = True
    ):
    """Submit a request to the agent and run any tools it calls."""
    # Initialize the chat history
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]

    while True:

        # Ask the agent what to do next
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        ).choices[0].message

        # Update the chat history with the agent's response
        messages.append({
            "role": "assistant",
            "content": response.content,
            "tool_calls": response.tool_calls
        })

        # If agent did not call any tools, we are done
        if not response.tool_calls:
            if verbose:
              print(f"\n⭐ The resulting set is: {MY_SET}")
            break

        # Execute all tool calls
        for tool_call in response.tool_calls:
            if verbose:
              print(f"\n🔧 The agent is calling a tool: "
                  f"{tool_call.function.name}"
                  f"({json.loads(tool_call.function.arguments)})")

            # Append the outcome to the message history
            outcome = execute(tool_call)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(outcome)
            })

## Let's test the agent!

Submit a request to the agent.

In [ ]:
MY_SET = set()
submit_request("Please add 'apples', 'oranges' and 'pears' to the set.")

Inspect the message history.

In [ ]:
submit_request("Please remove 'oranges' from the set.")

# 🗄️ Creating an MCP Server

Create the MCP server.

In [ ]:
%pip install --quiet mcp
from mcp.server import Server
from mcp.types import Tool, TextContent

server = Server("set-server")
print("✓ Server created")

Create an MCP wrapper for listing the available tools.

In [ ]:
async def list_tools() -> list[Tool]:
    """Return the list of available tools from our tools definition."""
    # Create an empty list to store MCP Tool objects
    mcp_tools = []

    # Convert each tool from our OpenAI format to MCP format
    for tool_def in tools:
        # Extract the function definition from the OpenAI tool format
        func_def = tool_def["function"]

        # Create an MCP Tool object with the same information
        mcp_tools.append(Tool(
            name=func_def["name"], # the function name
            description=func_def["description"], # what the tool does
            inputSchema=func_def["parameters"] # the JSON schema for parameters
        ))

    # Return the list of MCP Tool objects
    return mcp_tools

# Register the list_tools function with the server
server.list_tools()(list_tools)

Create an MCP wrapper for executing tools.

In [ ]:
from types import SimpleNamespace

async def call_tool(name: str, arguments: dict) -> list[TextContent]:
    """Handle MCP tool calls by delegating to our existing tools."""
    # Convert MCP format to the format expected by execute()
    function = SimpleNamespace(
        name=name,
        arguments=json.dumps(arguments)  # Convert dict to JSON string
    )
    tool_call = SimpleNamespace(function=function)

    # Execute the tool using the existing execute() function
    result = execute(tool_call)

    # Convert result to MCP response format
    return [TextContent(type="text", text=str(result))]

# Register the call_tool function with the server
server.call_tool()(call_tool)

Create a web application.

In [ ]:
# FastAPI is a framework for building REST APIs
%pip install --quiet fastapi
from mcp.server.sse import SseServerTransport
from fastapi import FastAPI, Request
from fastapi.responses import Response

# Create a FastAPI web application
app = FastAPI()

Expose a `POST` endpoint, used to handle incoming tool calls.

In [ ]:
# Create an SSE transport that will handle messages at the "/messages" path
sse = SseServerTransport("/messages")

# Mount the POST handler for receiving messages
# Clients send messages to http://host:port/messages
app.mount("/messages", sse.handle_post_message)

Expose a `GET` endpoint, used to establish the connection to the server.

In [ ]:
async def handle_sse(request: Request):
    """Handle incoming SSE connections from MCP clients."""
    # Connect the SSE transport to get read/write streams
    async with sse.connect_sse(
        request.scope, request.receive, request._send
    ) as (read_stream, write_stream):
        # Run the MCP server with these streams
        await server.run(
            read_stream,
            write_stream,
            server.create_initialization_options()
        )
    return Response()

# Register the GET endpoint with the FastAPI app
# Clients connect to http://host:port/sse to establish SSE connection
app.add_api_route("/sse", handle_sse, methods=["GET"])

Create a web server.

In [ ]:
# Uvicorn is a web server that handles HTTP requests and asynchronous code
%pip install --quiet uvicorn psutil

import uvicorn, time, atexit, os, psutil
from multiprocessing import Process


class ServerManager:
    _process = None

    @classmethod
    def _get_pid_on_port(cls, port):
        """Helper to find the PID of any process listening on the specified port."""
        for conn in psutil.net_connections(kind='inet'):
            if conn.laddr.port == port and conn.status == 'LISTEN':
                # Ensure we don't return the current Jupyter Kernel's PID
                if conn.pid != os.getpid():
                    return conn.pid
        return None


    @classmethod
    def start(cls, app_obj, port):
        """Idempotent start: Runs the server if port is free, otherwise informs the user."""
        existing_pid = cls._get_pid_on_port(port)
        
        if existing_pid:
            print(f"Server is already running (PID {existing_pid}) at http://127.0.0.1:{port}")
            return

        # Create and start the server in a separate background process
        cls._process = Process(
            target=uvicorn.run, 
            args=(app_obj,), 
            kwargs={'host': "0.0.0.0", 'port': port, 'log_level': "warning"}, 
            daemon=True
        )
        cls._process.start()
        
        # Wait to confirm the process stayed alive during startup
        time.sleep(3)
        if cls._process.is_alive():
            print(f"Server started (PID {cls._process.pid}) at http://127.0.0.1:{port}")
        else:
            print("Server failed to start.")


    @classmethod
    def stop(cls, port):
        """Idempotent stop: Requests shutdown and waits for the port to clear."""
        pid = cls._get_pid_on_port(port)
        if not pid:
            print(f"Port {port} is already clear.")
        else:
            try:
                proc = psutil.Process(pid)
                proc.terminate()
                # Return immediately when process dies (max 2 seconds)
                proc.wait(timeout=2)
                print(f"Server (PID {pid}) stopped.")
            except psutil.TimeoutExpired:
                print(f"Server (PID {pid}) is taking a long time to exit.")
        
        cls._process = None



Start the web server.

In [ ]:
# Configuration
server_port = 12345

# Execute Start
ServerManager.start(app, server_port)
server_url = f"http://127.0.0.1:{server_port}/sse"

# Register automatic cleanup for when the kernel or script exits
atexit.register(lambda: ServerManager.stop(server_port));

# 🤝 Creating an MCP Client

List available tools on the MCP server.

In [ ]:
from mcp import ClientSession
from mcp.client.sse import sse_client

# Utility function to list tools
async def list_mcp_tools():
    """Helper function to list available MCP tools."""
    async with sse_client(server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            available_tools = await session.list_tools()
            return available_tools

available_tools = await list_mcp_tools()
print("Available tools:", [t.name for t in available_tools.tools])

Test each tool, starting with an empty set.

In [ ]:
# Utility function to call a tool
async def call_mcp_tool(tool_name: str, arguments: dict = {}):
    """Helper function to call an MCP tool."""
    async with sse_client(server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            result = await session.call_tool(tool_name, arguments)
            return result

MY_SET = set()

Add "cherries" to the set.

In [ ]:
result = await call_mcp_tool("insertion_tool", {"s": "cherries"})
print(result.content[0].text)

Attempt to remove "bananas" from the set.

In [ ]:
result = await call_mcp_tool("removal_tool", {"s": "bananas"})
print(result.content[0].text)

Read all contents of the set.

In [ ]:
result = await call_mcp_tool("get_set_tool")
print(result.content[0].text)

# 🧠 Orchestration with LangGraph

In [ ]:
%pip install --quiet --upgrade "langchain" "langchain-openai" "langgraph" "langchain-mcp-adapters"

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Create MCP client that connects to your set-server
client = MultiServerMCPClient(
    {
        "set-server": {
            "transport": "sse",
            "url": f"http://localhost:{server_port}/sse",
        }
    }
)

Get available tools from the MCP server.

In [ ]:
tools_from_mcp = await client.get_tools()

for tool in tools_from_mcp:
    print(f"- {tool.name}: {tool.description}")

Create a LangGraph agent.

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

agent_executor = create_agent(
    ChatOpenAI(model="gpt-4o", temperature=0),
    tools_from_mcp,
)

The following function calls the LangGraph agent and prints the conversation history.

In [ ]:
async def submit_langgraph_request(user_prompt: str, verbose: bool = True):
    """Submit a request to the LangGraph agent and display the conversation."""
    from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

    # Run the agent with the system prompt and user's prompt
    result = await agent_executor.ainvoke({
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    })

    # Display the conversation if verbose
    if verbose:
        for message in result['messages']:

            # Print user message
            if isinstance(message, HumanMessage):
                print("\n👤 User: " + message.content)

            # Print agent message
            elif isinstance(message, AIMessage):
                if message.content:
                    print("\n🤖 Agent: " + message.content)

            # Print tool action
            elif isinstance(message, ToolMessage):
                # Extract text from content (handle both string and list of dicts)
                if isinstance(message.content, str):
                    outcome_text = message.content
                elif isinstance(message.content, list) and len(message.content) > 0:
                    # Extract 'text' field from the first item if it's a dict
                    outcome_text = message.content[0].get('text', str(message.content[0]))
                else:
                    outcome_text = str(message.content)

                print(f"\n🔧 {message.name}: {outcome_text}")

Let's test it!

In [ ]:
await submit_langgraph_request("Please add 'grapes', 'kiwi', and 'mango' to the set. Then display the set.")

# 👩🏻‍💻 Creating a Coding Assistant

Here's what the outdated application looks like:
https://dwoodlock.github.io/Metric-Treadmill-2017/

## Get the Code 

In [ ]:
CODE_DIRECTORY = "./Metric-Treadmill-2017"

In [ ]:
# Clone the Metric-Treadmill-2017 repo
from pathlib import Path
import shutil
import subprocess

path = Path("./Metric-Treadmill-2017")
if path.exists() and path.is_dir(): shutil.rmtree(path)

subprocess.run(["git", "clone", "--quiet", "https://github.com/dwoodlock/Metric-Treadmill-2017.git"], 
               check=True)

## Set up the Prompts

In [ ]:
system_prompt = """
You are a helpful assistant that can interact with a computer using tools.

You have access to the following tool:

- bash: Execute bash commands on the system

IMPORTANT: Every response must include:
1. A THOUGHT section explaining your reasoning and what you plan to do
2. A tool call to execute the bash command

Your THOUGHT should be in the text of your response, followed by the tool call.
Do not leave the text response empty - always explain your reasoning first.
"""

In [ ]:
# get the user prompt template 
import httpx
try:
    response = httpx.get("https://raw.githubusercontent.com/marta-manzin/agentic-shopping-assistant/refs/heads/main/prompts/coding-user-prompt-template.txt")
    response.raise_for_status()
    user_prompt_template = response.text
except httpx.HTTPError as e:
    print(f"Error: {e}")
    
print("\n".join(user_prompt_template.split("\n")[0:10]), "\n...") # print first 10 lines

## Updated the Tools - Add the Bash Tool

In [ ]:
def bash_tool(command: str) -> str:
    """Execute a bash command and return the result."""
    try:
        result = subprocess.run(
            command, shell=True, check=True,
            capture_output=True, text=True,
            cwd=CODE_DIRECTORY
        )
        return result.stdout if result.stdout else (
            "Command executed successfully with no output."
        )
    except subprocess.CalledProcessError as e:
        return f"Error executing command: {e.stderr}"

bash_description = """
Execute bash commands on the system.

## Useful command examples

### Create a new file:
cat <<'EOF' > newfile.py
import numpy as np
hello = "world"
print(hello)
EOF

### Edit files with sed:
IMPORTANT: You are on MacOS. Use `sed -i ''` instead of `sed -i`.

# Replace all occurrences
sed -i '' 's/old_string/new_string/g' filename.py

# Replace only first occurrence
sed -i '' 's/old_string/new_string/' filename.py

# Replace first occurrence on line 1
sed -i '' '1s/old_string/new_string/' filename.py

# Replace all occurrences in lines 1-10
sed -i '' '1,10s/old_string/new_string/g' filename.py

### View file content:
# View specific lines with numbers
nl -ba filename.py | sed -n '10,20p'

### Any other command you want to run
You can run any bash command including ls, cat, find, python, etc.
""".strip()

In [ ]:
# Add the Bash tool to our tools list.  (Check if it's been added already in case the cell is re-run)

if 'bash_tool' not in [tool['function']['name'] for tool in tools]: 
    tools.append(
{
    'type': 'function',
    'function': {
        'name': 'bash_tool',
        'description': bash_description,
        'parameters': {
            'type': 'object',
            'properties': {
                'command': {
                    'type': 'string',
                    'description': 'The bash command to execute.'
                }
            },
            'required': ['command']
        }
    }
})

In [ ]:
# check it
[tool['function']['name'] for tool in tools]

## Get the User Task

In [ ]:
user_prompt = """
I have a web app that I wrote many years ago that converts my running plans
into a metric system if I find myself on a treadmill internationally.
I’m concerned that it uses old libraries and old approaches and it’ll just
stop working one day.  Can you modernize this app for me.  I'd especially
like you to eliminate unneeded and deprecated libraries and use modern
language features and approaches.
""".strip()

In [ ]:
# Insert the user request into the user prompt template to create the full prompt
import platform
enhanced_user_prompt = user_prompt_template.replace(
    "{{user-prompt}}",
    user_prompt
).replace(
    "{{platform-uname}}",
    str(platform.uname()))

## Set up a new LangGraph agent

In [ ]:
tools_from_mcp = await client.get_tools()

In [ ]:
# Create a LangGraph agent using the updated tools
from langchain.agents import create_agent

agent_executor = create_agent(
    ChatOpenAI(model="gpt-4o", temperature=0),
    tools_from_mcp,
)

print("✓ LangGraph agent created")

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

async def run_agent_with_progress():
    iteration = 0
    async for event in agent_executor.astream({
        "messages": [{
            "role": "user", 
            "content": enhanced_user_prompt}]
    }):
        iteration = iteration + 1
        print("-" * 40, "\nIteration", iteration)
        if 'model' in event:
            messages = event['model']['messages']
        elif 'tools' in event:
            messages = event['tools']['messages']
        else:
            assert False, "Unimplemented event structure"
            
        for message in messages:
                if isinstance(message, AIMessage):
                    if isinstance(type(message.content), str):
                       print("AI: \033[34m" + message.content + "\033[0m")
                    elif isinstance(type(message.content), list):
                        for content_block in message.content:
                            if content_block['type'] == 'tool_use':
                                print("AI: \033[34m call", content_block['name'], content_block['input'], "\033[0m")
                            elif content_block['type'] == 'text':
                                print("AI: \033[34m", content_block['text'], "\033[0m")
                    else:
                        assert False, "Unimplemented message.content type"
                elif isinstance(message, ToolMessage):
                    print(f"Tool Result:\n\033[32m{message.content}\033[0m")
                else:
                    assert False, "Unimplemented message type"
    return event

In [ ]:
print("Here we go!")

In [ ]:
# result = await(run_agent_with_progress())

# Next Steps

<input type="checkbox"> Add a web search tool for the LangGraph agent \
<input type="checkbox"> Add a human-in-the-loop tool for the LangGraph agent \
<input type="checkbox"> Add support for item quantities (eg. 3 apples) \
<input type="checkbox"> Add a short section on recipe creation, to demonstrate the agent's features \
<input type="checkbox"> Add an agent design exercise, where we prompt audiences to draw a diagram \
<input type="checkbox"> Potentially add a short section on context management

# 🧹 Cleanup

Stop the MCP server.

In [ ]:
# Stop the MCP server using the ServerManager
ServerManager.stop(server_port)

# Thank you!

###